# 📊 Graph Database Fundamentals with Neo4j

## Welcome to Graph Databases!

In this notebook, you'll learn how to use **graph databases** to model and query complex, interconnected data. While vector databases excel at similarity search, graph databases are optimized for understanding and traversing **relationships** between entities.

## 🎯 What You'll Learn

- Core graph database concepts: nodes, relationships, and properties
- **Cypher query language** for pattern matching and graph traversal
- Building knowledge graphs from structured data
- Advanced pattern matching and multi-hop relationship queries
- When to use graph databases vs. vector databases in your AI applications

## 💼 Business Value: Why Graph Databases Matter

Graph databases power some of the world's most critical applications:

- **Social Networks**: LinkedIn uses graphs to show "how you're connected" to other professionals
- **Fraud Detection**: Banks detect fraud rings by analyzing transaction networks in real-time
- **Recommendation Systems**: Netflix and Amazon use graphs to find "people like you" and recommend content
- **Knowledge Management**: Enterprises build knowledge graphs to connect documents, people, projects, and expertise
- **Supply Chain Optimization**: Companies model complex supply networks to identify bottlenecks and risks

Unlike traditional databases that struggle with complex joins across multiple tables, graph databases are designed to traverse relationships at lightning speed—even across millions of connections.

## 🔗 How This Connects to Your Previous Learning

You've already learned about:
- **Vector databases**: Great for similarity search ("find documents similar to this")
- **RAG systems**: Retrieve relevant context for LLM queries
- **LlamaIndex**: Orchestrate data ingestion and retrieval

Graph databases complement these tools by adding **relationship intelligence**. In the next notebook, you'll combine graph databases with vector search to create **GraphRAG**—the next evolution of retrieval-augmented generation.

Let's get started! 🚀

---
## 📚 Part 1: Theory - What Are Graph Databases?

### Understanding Graph Databases

A **graph database** stores data as a network of interconnected nodes and relationships, rather than rows and tables. This structure mirrors how we naturally think about connected information: people know people, products belong to categories, documents cite other documents.

Traditional **relational databases** require complex JOIN operations to traverse relationships. As you add more joins ("find friends of friends of friends"), performance degrades exponentially. **Document databases** like MongoDB store data as self-contained documents, but struggle to represent complex relationships efficiently.

Graph databases solve this by making relationships **first-class citizens**. Relationships are stored directly alongside the data, enabling constant-time traversal regardless of database size. A query that finds "friends within 3 degrees of separation" executes in milliseconds even with millions of users.

### 🔑 Key Terminology

Every graph database is built from three fundamental elements:

1. **Nodes** (also called vertices): Represent entities in your domain
   - Examples: Person, Company, Product, Document
   - Have **labels** to categorize them (e.g., `:Employee`, `:Department`)
   - Contain **properties** with key-value data (e.g., `name: "Sarah"`, `age: 35`)

2. **Relationships** (also called edges): Connect nodes and represent how they're related
   - Examples: WORKS_FOR, PURCHASED, KNOWS, MANAGES
   - Always have a **direction** (though you can query in either direction)
   - Also contain **properties** (e.g., `since: 2020`, `strength: 0.8`)

3. **Properties**: Key-value pairs that store data on nodes and relationships
   - Examples: `salary: 75000`, `created_at: "2024-01-15"`, `priority: "high"`

### 🎯 When to Use Graph Databases vs. Vector Databases

Choose **graph databases** when:
- Your data has complex, multi-hop relationships (organizational hierarchies, social networks)
- You need to answer "how are these connected?" questions
- Relationships are as important as the entities themselves
- You're detecting patterns across connections (fraud rings, influence networks)

Choose **vector databases** when:
- You need semantic similarity search ("find documents similar to this query")
- Your data is unstructured (text, images, audio)
- Relationships are less important than content similarity

The best systems use **both**: vector search to find relevant content, graph traversal to understand context and relationships. You'll learn this hybrid approach in Notebook 2!

### 🌟 Real-World Use Cases

1. **LinkedIn Connections**: "How are you connected to this person?" requires traversing the professional network graph
2. **Fraud Detection**: Banks analyze transaction graphs to find suspicious patterns—multiple accounts transferring to the same destination
3. **Recommendation Engines**: "Users who bought X also bought Y" is a graph traversal problem
4. **Drug Discovery**: Pharmaceutical companies use knowledge graphs to find relationships between proteins, diseases, and compounds
5. **IT Operations**: Map dependencies between services to understand cascade failures

### 💡 Key Point: The Power of Relationship Traversal

In a relational database, finding "friends of friends" requires multiple self-joins that get exponentially slower. In a graph database, you simply follow the relationships—performance stays constant regardless of how many "hops" you traverse. This is called **index-free adjacency**, and it's what makes graph databases uniquely powerful for connected data.



---
## ⚙️ Part 2: Setup - Installing Neo4j in Google Colab

Neo4j is the world's leading graph database. We'll install **Neo4j Community Edition** directly in this Colab environment—no external accounts or services needed!

### What We're Installing

- **Neo4j database**: Graph database engine that runs locally in this Colab VM
- **Python driver**: Official Neo4j Python library for executing Cypher queries
- **Supporting libraries**: pandas, matplotlib, networkx for data manipulation and visualization

This installation takes about 30 seconds. Let's begin! ⚡

In [ ]:
# 📦 Install Neo4j Community Edition in Colab
# Simple approach: Download tarball, extract, and run directly

import time
import os

print("🔄 Step 1: Installing Java (required for Neo4j)...")
!apt-get update -qq > /dev/null 2>&1
!apt-get install -y default-jre wget > /dev/null 2>&1
print("✅ Java installed!\n")

print("📥 Step 2: Downloading Neo4j Community Edition 4.4.36...")
!wget -q https://dist.neo4j.org/neo4j-community-4.4.36-unix.tar.gz
print("✅ Downloaded!\n")

print("📦 Step 3: Extracting Neo4j...")
!tar -xf neo4j-community-4.4.36-unix.tar.gz
!mv neo4j-community-4.4.36 neo4j
print("✅ Extracted to 'neo4j' folder!\n")

print("🔐 Step 4: Setting initial password...")
# Set the initial password using neo4j-admin
!neo4j/bin/neo4j-admin set-initial-password password123
print("✅ Password set to: password123\n")

print("🚀 Step 5: Starting Neo4j server...")
# Start Neo4j in the background
!neo4j/bin/neo4j start
print("✅ Neo4j starting...\n")

print("📚 Step 6: Installing Python drivers...")
!pip install -q neo4j py2neo
print("✅ Python drivers installed!\n")

print("⏳ Step 7: Waiting for Neo4j to be fully ready...")
# Wait for Neo4j to be ready (check if port 7687 is listening)
import socket
max_attempts = 30
for attempt in range(max_attempts):
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)
        result = sock.connect_ex(('127.0.0.1', 7687))
        sock.close()
        if result == 0:
            print(f"✅ Neo4j is ready! (took {attempt * 2} seconds)\n")
            break
    except:
        pass

    if attempt % 5 == 0 and attempt > 0:
        print(f"   Still waiting... ({attempt}/{max_attempts})")

    time.sleep(2)

print("=" * 60)
print("📊 NEO4J CONNECTION DETAILS")
print("=" * 60)
print("URI:      bolt://localhost:7687")
print("Username: neo4j")
print("Password: password123")
print("=" * 60)
print("\n🎉 Setup complete! Ready to connect.\n")

In [ ]:
# 📦 Install additional dependencies for data manipulation and visualization

print("📥 Installing additional libraries...")
!pip install -q pandas matplotlib networkx python-louvain

# Suppress unnecessary warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

print("✅ All dependencies installed!")

In [ ]:
# 📚 Import all necessary libraries

# Neo4j driver for database connection and queries
from neo4j import GraphDatabase

# Data manipulation and analysis
import pandas as pd
import json
from typing import List, Dict, Any, Optional

# Visualization libraries
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.patches import Patch

# Utilities
import time
from pprint import pprint

print("✅ All libraries imported successfully!")

In [ ]:
# 🔗 Configure Neo4j connection parameters

NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password123"

# Create the database driver (connection pool)
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

print("✅ Neo4j driver created!")

In [ ]:
# 🧪 Test Neo4j connection with retry logic

def test_neo4j_connection(max_retries=5, retry_delay=3):
    """
    Verify that we can connect to Neo4j and execute a simple query.
    This function tests the connection with retry logic for robustness.

    Args:
        max_retries: Maximum number of connection attempts
        retry_delay: Seconds to wait between retries
    """
    for attempt in range(1, max_retries + 1):
        try:
            print(f"🔄 Connection attempt {attempt}/{max_retries}...")

            # Open a session and run a simple query
            with driver.session() as session:
                # Count all nodes in the database (should be 0 for a fresh install)
                result = session.run("MATCH (n) RETURN count(n) as node_count")
                record = result.single()
                node_count = record["node_count"]

                print("\n✅ Successfully connected to Neo4j!")
                print(f"📊 Current node count: {node_count}")
                print("🎉 Database is ready to use!")
                return True

        except Exception as e:
            print(f"❌ Attempt {attempt} failed: {str(e)[:100]}")

            if attempt < max_retries:
                print(f"⏳ Waiting {retry_delay} seconds before retry...")
                time.sleep(retry_delay)
            else:
                print("\n❌ All connection attempts failed!")
                print("\n🔧 Troubleshooting steps:")
                print("   1. Check if Neo4j is running: !neo4j status")
                print("   2. Try restarting Neo4j: !neo4j restart")
                print("   3. Wait a bit longer and run this cell again")
                print("   4. Check Neo4j logs: !cat /var/log/neo4j/neo4j.log | tail -50")
                return False

# Run the connection test with retries
test_neo4j_connection()

In [ ]:
# 🔧 TROUBLESHOOTING: Run this cell ONLY if connection failed
# Uncomment the lines below if you need to restart Neo4j

# print("🔍 Checking Neo4j status...")
# !neo4j/bin/neo4j status

# print("\n🔄 Attempting to restart Neo4j...")
# !neo4j/bin/neo4j restart

# print("\n⏳ Waiting 30 seconds for Neo4j to fully initialize...")
# import time
# time.sleep(30)

# print("\n✅ Neo4j restart complete!")
# print("📊 Now try running the connection test cell above again.")

# # Optionally check logs for errors
# print("\n📋 Recent Neo4j logs (last 20 lines):")
# !tail -20 neo4j/logs/neo4j.log 2>/dev/null || echo "Could not access logs (logs may be in a different location)"

print("⚠️ This cell is commented out by default.")
print("Only uncomment and run if the connection test above failed.")

---
## 🔤 Part 3: Cypher Basics - The Graph Query Language

### Introduction to Cypher

**Cypher** is Neo4j's query language for working with graphs. If you know SQL, you'll find Cypher familiar but more intuitive for connected data. Instead of thinking about tables and joins, you think about **patterns** in the graph.

### Basic Syntax Patterns

Cypher uses ASCII art to represent graph patterns:

```cypher
()                    // A node
(p:Person)            // A node with label 'Person', aliased as 'p'
(p:Person {name: "Sarah"})  // A node with properties

-->                   // A relationship (directed)
-[r]->                // A relationship, aliased as 'r'
-[:KNOWS]->           // A relationship with type 'KNOWS'
-[:KNOWS {since: 2020}]->  // A relationship with properties

(p1:Person)-[:KNOWS]->(p2:Person)  // A complete pattern
```

### Core Cypher Commands

- **CREATE**: Add new nodes and relationships to the graph
- **MATCH**: Find patterns in the graph (like SQL's SELECT)
- **WHERE**: Filter results based on conditions
- **RETURN**: Specify what data to return
- **DELETE**: Remove nodes and relationships (use DETACH DELETE to remove relationships too)
- **MERGE**: Create if doesn't exist, or match if it does (prevents duplicates)

### 💡 Key Point: Cypher's Pattern-Matching Power

The magic of Cypher is in pattern matching. Instead of writing complex joins, you simply **draw the pattern** you're looking for using ASCII art. The database figures out how to traverse the graph efficiently. Want to find "people who work with people who know each other"? Just write the pattern:

```cypher
(p1:Person)-[:WORKS_WITH]->(p2:Person)-[:KNOWS]->(p3:Person)
```

Let's try some examples!

In [ ]:
# 🛠️ Helper function to execute Cypher queries

def run_query(query: str, parameters: Optional[Dict[str, Any]] = None) -> List[Dict[str, Any]]:
    """
    Execute a Cypher query and return results as a list of dictionaries.

    Args:
        query: Cypher query string
        parameters: Optional dictionary of parameters for the query

    Returns:
        List of result records as dictionaries
    """
    try:
        with driver.session() as session:
            result = session.run(query, parameters or {})
            # Convert results to list of dictionaries
            records = [dict(record) for record in result]
            return records
    except Exception as e:
        print(f"❌ Query execution failed: {str(e)}")
        print(f"Query: {query}")
        return []

print("✅ Helper function created!")

 ⚠️ Important: The Examples Below Are TEMPORARY. Before we build our real company knowledge graph, let's learn Cypher syntax with simple examples. We'll create some test nodes and relationships, experiment with queries, then **delete everything** —we're just learning the syntax!

 🎯 What We'll Do:
 1. ✏️ **Create** simple test nodes (Alice, Bob, TechCorp)
 2. 🔍 **Practice** querying them with various Cypher commands
 3. 🗑️ **Delete everything** to clean up
 4. 🏗️ **Then** build our real knowledge graph in Part 4

 **Why are we doing this?** You need to understand Cypher syntax before working with real data. These throwaway examples let you experiment without worrying about messing up the real knowledge graph. Let's practice! 👇

In [ ]:
# 1️⃣ CREATE a single node
print("1️⃣ Creating a Person node...")

query = """
CREATE (p:Person {name: "Alice", age: 30, role: "Data Scientist"})
RETURN p.name as name, p.age as age, p.role as role
"""
result = run_query(query)
print(f"✅ Created: {result[0]}\n")

In [ ]:
# 2️⃣ CREATE multiple nodes with different labels
print("2️⃣ Creating multiple nodes...")

query = """
CREATE (p:Person {name: "Bob", age: 35}),
       (c:Company {name: "TechCorp", industry: "Software"}),
       (s:Skill {name: "Python", category: "Programming"})
RETURN 'Nodes created' as status
"""
result = run_query(query)
print(f"✅ {result[0]['status']}\n")

In [ ]:
# 3️⃣ CREATE nodes with relationships
print("3️⃣ Creating nodes with relationships...")

query = """
CREATE (alice:Person {name: "Alice"})-[:WORKS_FOR {since: 2020}]->(company:Company {name: "DataCo"})
RETURN alice.name as person, company.name as company
"""
result = run_query(query)
print(f"✅ {result[0]['person']} works for {result[0]['company']}\n")

In [ ]:
# 4️⃣ MATCH and return nodes
print("4️⃣ Finding all Person nodes...")

query = """
MATCH (p:Person)
RETURN p.name as name, p.age as age
ORDER BY p.name
"""
results = run_query(query)
print(f"✅ Found {len(results)} people:")
for person in results:
    print(f"   - {person['name']}, age: {person.get('age', 'N/A')}")
print()

In [ ]:
# 5️⃣ MATCH patterns with relationships
print("5️⃣ Finding who works for which companies...")

query = """
MATCH (p:Person)-[r:WORKS_FOR]->(c:Company)
RETURN p.name as employee, c.name as company, r.since as since
"""
results = run_query(query)
if results:
    print(f"✅ Found {len(results)} employment relationships:")
    for rel in results:
        print(f"   - {rel['employee']} → {rel['company']} (since {rel.get('since', 'unknown')})")
else:
    print("No employment relationships found.")
print()

In [ ]:
# 6️⃣ Use WHERE to filter results
print("6️⃣ Finding people over age 30...")

query = """
MATCH (p:Person)
WHERE p.age > 30
RETURN p.name as name, p.age as age
ORDER BY p.age DESC
"""
results = run_query(query)
if results:
    print(f"✅ Found {len(results)} people over 30:")
    for person in results:
        print(f"   - {person['name']}: {person['age']} years old")
else:
    print("No people over 30 found.")
print()

In [ ]:
# 7️⃣ Clean up test data
print("7️⃣ Cleaning up test data...")

query = """
MATCH (n)
DETACH DELETE n
"""
run_query(query)
print("✅ All nodes and relationships deleted!\n")

# Verify database is empty
verify_query = "MATCH (n) RETURN count(n) as count"
result = run_query(verify_query)
print(f"📊 Current node count: {result[0]['count']}")

### 🎯 Cypher Basics - Key Takeaways

- Cypher uses **ASCII art** to represent graph patterns: `(node)-[:RELATIONSHIP]->(node)`
- **CREATE** adds new data, **MATCH** finds existing patterns
- Use **DETACH DELETE** to remove nodes and their relationships
- Properties are specified in curly braces: `{name: "Alice", age: 30}`
- Pattern matching is intuitive—just draw what you're looking for!
- Cypher reads like English: "Match this pattern, where this condition, return that"

Now that you understand basic Cypher, let's build a real knowledge graph! 🚀

---
## 🏢 Part 4: Building a Knowledge Graph

### Use Case: Company Organizational Knowledge Graph

We'll build a knowledge graph representing a company's organizational structure. This is a common real-world use case where graph databases excel.

### 📊 Our Graph Schema

**Entities (Nodes):**
- **Employee**: People who work at the company
- **Department**: Organizational units
- **Project**: Work initiatives
- **Skill**: Technical and professional competencies

**Relationships:**
- **(Employee)-[:WORKS_IN]->(Department)**: Where someone works
- **(Employee)-[:MANAGES]->(Employee)**: Organizational hierarchy
- **(Employee)-[:ASSIGNED_TO]->(Project)**: Project assignments
- **(Employee)-[:HAS_SKILL]->(Skill)**: Professional competencies

### 💡 Why This Structure Matters

This graph enables powerful queries like:
- "Who has Python skills in the Data Science department?"
- "Show me the reporting chain from employee to CEO"
- "Find colleagues who work on similar projects"
- "Which departments collaborate on projects most?"

Let's create realistic mock data for our company!

### 📝 Important Note: Manual Data Creation for Learning

In this notebook, we're **manually** creating mock data and then programmatically inserting it into Neo4j. This is intentional for educational purposes—it helps you:

- ✅ **Understand what happens "under the hood"** when data enters a graph database
- ✅ **Learn graph modeling** (deciding what should be nodes vs. relationships)
- ✅ **See the schema design process** step by step
- ✅ **Practice writing Cypher CREATE statements**

### 🏭 What Happens in the Real World (covered in the following notebook)

In production and in our next notebook, **node and relationship creation is automated**:

- 🤖 **Bulk CSV imports**: Load millions of records from files automatically
- 🤖 **ETL pipelines**: Extract data from other databases and transform it into graphs
- 🤖 **API integrations**: Real-time data from user actions, IoT sensors, etc.
- 🤖 **LLM-powered extraction**: Use AI to automatically extract entities and relationships from unstructured text (this is what we'll do in Notebook 2!)

### 🎯 The Key Distinction

- **You design the model once** (manual thinking): "Employees should be nodes, WORKS_IN should be a relationship"
- **The data insertion is automated** (computer does it): Creating 100,000 employee nodes from a CSV file

To insert data into Neo4j, we will create several functions. Let's get started!

In [ ]:
# 📝 Generate comprehensive mock data for our company knowledge graph

# Employees with realistic details
employees = [
    {"id": "emp001", "name": "Sarah Chen", "title": "VP of Data Science", "years_experience": 12, "email": "sarah.chen@company.com"},
    {"id": "emp002", "name": "Marcus Johnson", "title": "Senior ML Engineer", "years_experience": 7, "email": "marcus.j@company.com"},
    {"id": "emp003", "name": "Priya Patel", "title": "Data Scientist", "years_experience": 4, "email": "priya.p@company.com"},
    {"id": "emp004", "name": "James Liu", "title": "ML Engineer", "years_experience": 5, "email": "james.liu@company.com"},
    {"id": "emp005", "name": "Elena Rodriguez", "title": "Data Analyst", "years_experience": 3, "email": "elena.r@company.com"},
    {"id": "emp006", "name": "David Kim", "title": "VP of Engineering", "years_experience": 15, "email": "david.kim@company.com"},
    {"id": "emp007", "name": "Amy Zhang", "title": "Senior Backend Engineer", "years_experience": 8, "email": "amy.zhang@company.com"},
    {"id": "emp008", "name": "Carlos Santos", "title": "Frontend Engineer", "years_experience": 5, "email": "carlos.s@company.com"},
    {"id": "emp009", "name": "Lisa Anderson", "title": "DevOps Engineer", "years_experience": 6, "email": "lisa.a@company.com"},
    {"id": "emp010", "name": "Mohammed Ali", "title": "Backend Engineer", "years_experience": 4, "email": "mohammed.a@company.com"},
    {"id": "emp011", "name": "Sophie Martin", "title": "Product Manager", "years_experience": 9, "email": "sophie.m@company.com"},
    {"id": "emp012", "name": "Raj Sharma", "title": "Product Designer", "years_experience": 6, "email": "raj.s@company.com"},
    {"id": "emp013", "name": "Nina Williams", "title": "UX Researcher", "years_experience": 5, "email": "nina.w@company.com"},
    {"id": "emp014", "name": "Alex Turner", "title": "CEO", "years_experience": 20, "email": "alex.turner@company.com"},
    {"id": "emp015", "name": "Jennifer Lee", "title": "CTO", "years_experience": 18, "email": "jennifer.lee@company.com"},
    {"id": "emp016", "name": "Robert Brown", "title": "Senior Data Scientist", "years_experience": 8, "email": "robert.b@company.com"},
    {"id": "emp017", "name": "Maria Garcia", "title": "Junior ML Engineer", "years_experience": 2, "email": "maria.g@company.com"},
    {"id": "emp018", "name": "Tom Wilson", "title": "Data Engineer", "years_experience": 5, "email": "tom.w@company.com"},
]

# Departments with budgets
departments = [
    {"id": "dept001", "name": "Data Science", "budget": 2500000, "location": "Building A"},
    {"id": "dept002", "name": "Engineering", "budget": 5000000, "location": "Building B"},
    {"id": "dept003", "name": "Product", "budget": 1500000, "location": "Building A"},
    {"id": "dept004", "name": "Executive", "budget": 3000000, "location": "Building C"},
]

# Projects with status and priority
projects = [
    {"id": "proj001", "name": "Customer Churn Prediction", "status": "active", "priority": "high", "budget": 500000},
    {"id": "proj002", "name": "Recommendation Engine v2", "status": "active", "priority": "high", "budget": 750000},
    {"id": "proj003", "name": "Real-time Analytics Dashboard", "status": "active", "priority": "medium", "budget": 300000},
    {"id": "proj004", "name": "Mobile App Redesign", "status": "planning", "priority": "medium", "budget": 400000},
    {"id": "proj005", "name": "Data Pipeline Optimization", "status": "active", "priority": "high", "budget": 350000},
    {"id": "proj006", "name": "NLP Chatbot", "status": "completed", "priority": "low", "budget": 200000},
    {"id": "proj007", "name": "Fraud Detection System", "status": "active", "priority": "critical", "budget": 900000},
    {"id": "proj008", "name": "Infrastructure Migration", "status": "active", "priority": "high", "budget": 600000},
    {"id": "proj009", "name": "A/B Testing Platform", "status": "planning", "priority": "medium", "budget": 250000},
    {"id": "proj010", "name": "Computer Vision API", "status": "active", "priority": "medium", "budget": 450000},
]

# Skills taxonomy
skills = [
    {"name": "Python", "category": "Programming", "level": "essential"},
    {"name": "TensorFlow", "category": "ML Framework", "level": "advanced"},
    {"name": "PyTorch", "category": "ML Framework", "level": "advanced"},
    {"name": "SQL", "category": "Database", "level": "essential"},
    {"name": "Neo4j", "category": "Database", "level": "specialized"},
    {"name": "AWS", "category": "Cloud", "level": "essential"},
    {"name": "Docker", "category": "DevOps", "level": "essential"},
    {"name": "Kubernetes", "category": "DevOps", "level": "advanced"},
    {"name": "React", "category": "Frontend", "level": "essential"},
    {"name": "Node.js", "category": "Backend", "level": "essential"},
    {"name": "NLP", "category": "AI Specialty", "level": "specialized"},
    {"name": "Computer Vision", "category": "AI Specialty", "level": "specialized"},
    {"name": "Scikit-learn", "category": "ML Framework", "level": "essential"},
    {"name": "Apache Spark", "category": "Big Data", "level": "advanced"},
    {"name": "Product Strategy", "category": "Business", "level": "specialized"},
]

# Relationships: Employee works in Department
works_in = [
    ("emp001", "dept001"), ("emp002", "dept001"), ("emp003", "dept001"),
    ("emp004", "dept001"), ("emp005", "dept001"), ("emp016", "dept001"),
    ("emp017", "dept001"), ("emp018", "dept001"),
    ("emp006", "dept002"), ("emp007", "dept002"), ("emp008", "dept002"),
    ("emp009", "dept002"), ("emp010", "dept002"),
    ("emp011", "dept003"), ("emp012", "dept003"), ("emp013", "dept003"),
    ("emp014", "dept004"), ("emp015", "dept004"),
]

# Relationships: Manager manages Employee
manages = [
    ("emp014", "emp015"),  # CEO -> CTO
    ("emp014", "emp001"),  # CEO -> VP Data Science
    ("emp014", "emp006"),  # CEO -> VP Engineering
    ("emp015", "emp006"),  # CTO -> VP Engineering
    ("emp001", "emp002"),  # VP DS -> Senior ML Engineer
    ("emp001", "emp016"),  # VP DS -> Senior Data Scientist
    ("emp002", "emp003"),  # Senior ML -> Data Scientist
    ("emp002", "emp004"),  # Senior ML -> ML Engineer
    ("emp016", "emp017"),  # Senior DS -> Junior ML Engineer
    ("emp001", "emp018"),  # VP DS -> Data Engineer
    ("emp001", "emp005"),  # VP DS -> Data Analyst
    ("emp006", "emp007"),  # VP Eng -> Senior Backend
    ("emp007", "emp010"),  # Senior Backend -> Backend Engineer
    ("emp006", "emp008"),  # VP Eng -> Frontend Engineer
    ("emp006", "emp009"),  # VP Eng -> DevOps Engineer
    ("emp011", "emp012"),  # PM -> Product Designer
    ("emp011", "emp013"),  # PM -> UX Researcher
]

# Relationships: Employee assigned to Project
assigned_to = [
    ("emp001", "proj001"), ("emp003", "proj001"), ("emp005", "proj001"),
    ("emp002", "proj002"), ("emp004", "proj002"), ("emp016", "proj002"),
    ("emp018", "proj003"), ("emp007", "proj003"), ("emp005", "proj003"),
    ("emp008", "proj004"), ("emp012", "proj004"), ("emp013", "proj004"),
    ("emp018", "proj005"), ("emp009", "proj005"), ("emp007", "proj005"),
    ("emp004", "proj006"), ("emp003", "proj006"),
    ("emp002", "proj007"), ("emp016", "proj007"), ("emp004", "proj007"),
    ("emp009", "proj008"), ("emp007", "proj008"), ("emp010", "proj008"),
    ("emp011", "proj009"), ("emp012", "proj009"),
    ("emp003", "proj010"), ("emp017", "proj010"),
]

# Relationships: Employee has Skill
has_skill = [
    # Sarah Chen - VP Data Science
    ("emp001", "Python"), ("emp001", "TensorFlow"), ("emp001", "AWS"), ("emp001", "SQL"),
    # Marcus Johnson - Senior ML Engineer
    ("emp002", "Python"), ("emp002", "TensorFlow"), ("emp002", "PyTorch"), ("emp002", "Docker"),
    # Priya Patel - Data Scientist
    ("emp003", "Python"), ("emp003", "Scikit-learn"), ("emp003", "SQL"), ("emp003", "NLP"),
    # James Liu - ML Engineer
    ("emp004", "Python"), ("emp004", "PyTorch"), ("emp004", "Docker"), ("emp004", "NLP"),
    # Elena Rodriguez - Data Analyst
    ("emp005", "Python"), ("emp005", "SQL"), ("emp005", "Scikit-learn"),
    # David Kim - VP Engineering
    ("emp006", "Python"), ("emp006", "AWS"), ("emp006", "Kubernetes"), ("emp006", "Docker"),
    # Amy Zhang - Senior Backend Engineer
    ("emp007", "Python"), ("emp007", "Node.js"), ("emp007", "SQL"), ("emp007", "Docker"), ("emp007", "AWS"),
    # Carlos Santos - Frontend Engineer
    ("emp008", "React"), ("emp008", "Node.js"),
    # Lisa Anderson - DevOps Engineer
    ("emp009", "Docker"), ("emp009", "Kubernetes"), ("emp009", "AWS"),
    # Mohammed Ali - Backend Engineer
    ("emp010", "Python"), ("emp010", "Node.js"), ("emp010", "SQL"), ("emp010", "Docker"),
    # Sophie Martin - Product Manager
    ("emp011", "Product Strategy"), ("emp011", "SQL"),
    # Raj Sharma - Product Designer
    ("emp012", "React"),
    # Nina Williams - UX Researcher
    ("emp013", "Product Strategy"),
    # Robert Brown - Senior Data Scientist
    ("emp016", "Python"), ("emp016", "TensorFlow"), ("emp016", "Scikit-learn"), ("emp016", "SQL"),
    # Maria Garcia - Junior ML Engineer
    ("emp017", "Python"), ("emp017", "TensorFlow"), ("emp017", "Computer Vision"),
    # Tom Wilson - Data Engineer
    ("emp018", "Python"), ("emp018", "SQL"), ("emp018", "Apache Spark"), ("emp018", "AWS"),
]

print(f"✅ Mock data generated!")
print(f"   📊 {len(employees)} employees")
print(f"   🏢 {len(departments)} departments")
print(f"   📁 {len(projects)} projects")
print(f"   🎯 {len(skills)} skills")
print(f"   🔗 {len(works_in) + len(manages) + len(assigned_to) + len(has_skill)} relationships")

In [ ]:
# 🧹 Clear existing data to start fresh

print("🧹 Clearing database...")
run_query("MATCH (n) DETACH DELETE n")
print("✅ Database cleared!\n")

In [ ]:
# 🏗️ Create Employee nodes

def create_employees():
    """
    Create all Employee nodes in the graph.
    Uses UNWIND for efficient batch creation.
    """
    query = """
    UNWIND $employees AS emp
    CREATE (e:Employee {
        id: emp.id,
        name: emp.name,
        title: emp.title,
        years_experience: emp.years_experience,
        email: emp.email
    })
    """
    run_query(query, {"employees": employees})
    print(f"✅ Created {len(employees)} Employee nodes")

create_employees()

In [ ]:
# 🏢 Create Department nodes

def create_departments():
    """
    Create all Department nodes in the graph.
    """
    query = """
    UNWIND $departments AS dept
    CREATE (d:Department {
        id: dept.id,
        name: dept.name,
        budget: dept.budget,
        location: dept.location
    })
    """
    run_query(query, {"departments": departments})
    print(f"✅ Created {len(departments)} Department nodes")

create_departments()

In [ ]:
# 📁 Create Project nodes

def create_projects():
    """
    Create all Project nodes in the graph.
    """
    query = """
    UNWIND $projects AS proj
    CREATE (p:Project {
        id: proj.id,
        name: proj.name,
        status: proj.status,
        priority: proj.priority,
        budget: proj.budget
    })
    """
    run_query(query, {"projects": projects})
    print(f"✅ Created {len(projects)} Project nodes")

create_projects()

In [ ]:
# 🎯 Create Skill nodes

def create_skills():
    """
    Create all Skill nodes in the graph.
    """
    query = """
    UNWIND $skills AS skill
    CREATE (s:Skill {
        name: skill.name,
        category: skill.category,
        level: skill.level
    })
    """
    run_query(query, {"skills": skills})
    print(f"✅ Created {len(skills)} Skill nodes")

create_skills()

In [ ]:
# 🔗 Create all relationships

def create_relationships():
    """
    Create all relationships between nodes.
    This establishes the graph structure.
    """
    # WORKS_IN relationships
    works_in_query = """
    UNWIND $relationships AS rel
    MATCH (e:Employee {id: rel.emp_id})
    MATCH (d:Department {id: rel.dept_id})
    CREATE (e)-[:WORKS_IN]->(d)
    """
    works_in_data = [{"emp_id": e, "dept_id": d} for e, d in works_in]
    run_query(works_in_query, {"relationships": works_in_data})
    print(f"✅ Created {len(works_in)} WORKS_IN relationships")

    # MANAGES relationships
    manages_query = """
    UNWIND $relationships AS rel
    MATCH (manager:Employee {id: rel.manager_id})
    MATCH (report:Employee {id: rel.report_id})
    CREATE (manager)-[:MANAGES]->(report)
    """
    manages_data = [{"manager_id": m, "report_id": r} for m, r in manages]
    run_query(manages_query, {"relationships": manages_data})
    print(f"✅ Created {len(manages)} MANAGES relationships")

    # ASSIGNED_TO relationships
    assigned_query = """
    UNWIND $relationships AS rel
    MATCH (e:Employee {id: rel.emp_id})
    MATCH (p:Project {id: rel.proj_id})
    CREATE (e)-[:ASSIGNED_TO]->(p)
    """
    assigned_data = [{"emp_id": e, "proj_id": p} for e, p in assigned_to]
    run_query(assigned_query, {"relationships": assigned_data})
    print(f"✅ Created {len(assigned_to)} ASSIGNED_TO relationships")

    # HAS_SKILL relationships
    skill_query = """
    UNWIND $relationships AS rel
    MATCH (e:Employee {id: rel.emp_id})
    MATCH (s:Skill {name: rel.skill_name})
    CREATE (e)-[:HAS_SKILL]->(s)
    """
    skill_data = [{"emp_id": e, "skill_name": s} for e, s in has_skill]
    run_query(skill_query, {"relationships": skill_data})
    print(f"✅ Created {len(has_skill)} HAS_SKILL relationships")

create_relationships()

In [ ]:
# 🔍 Verify data load

print("📊 Verifying knowledge graph structure...\n")

# Count nodes by label
node_query = """
MATCH (n)
RETURN labels(n)[0] as label, count(*) as count
ORDER BY count DESC
"""
node_results = run_query(node_query)
print("📍 Nodes by type:")
for row in node_results:
    print(f"   {row['label']}: {row['count']}")

# Count relationships by type
rel_query = """
MATCH ()-[r]->()
RETURN type(r) as relationship, count(*) as count
ORDER BY count DESC
"""
rel_results = run_query(rel_query)
print("\n🔗 Relationships by type:")
for row in rel_results:
    print(f"   {row['relationship']}: {row['count']}")

print("\n✅ Knowledge graph successfully created!")

---
## 🔍 Part 5: Pattern Matching & Traversal

### The Power of Pattern Matching

Now that our knowledge graph is built, we can unleash the real power of graph databases: **pattern matching** and **relationship traversal**. Unlike SQL joins that get slower with complexity, graph traversals maintain constant performance.

### What We'll Explore

- **Single-hop queries**: Direct relationships (employee → department)
- **Multi-hop queries**: Chained relationships (employee → project → other employees)
- **Variable-length paths**: Find connections within N degrees
- **Aggregations**: Count, collect, and analyze patterns
- **Pattern combinations**: Complex queries matching multiple patterns simultaneously

### 💡 Key Point: Traversal Performance

Neo4j's **index-free adjacency** means each node physically stores references to its connected relationships. Traversing from one node to another is a simple pointer lookup—no index searches required. This is why graph databases can traverse millions of relationships in milliseconds, while relational databases struggle with just a few joins.

Let's explore with practical queries! 🚀

In [ ]:
# 1️⃣ Find all employees in a specific department
print("1️⃣ Who works in the Data Science department?\n")

query = """
MATCH (e:Employee)-[:WORKS_IN]->(d:Department {name: "Data Science"})
RETURN e.name as name, e.title as title, e.years_experience as experience
ORDER BY e.years_experience DESC
"""
results = run_query(query)
print(f"📊 Found {len(results)} Data Science employees:\n")
for emp in results:
    print(f"   • {emp['name']} - {emp['title']} ({emp['experience']} years)")
print()

In [ ]:
# 2️⃣ Find organizational hierarchy (who manages whom)
print("2️⃣ Organizational reporting structure:\n")

query = """
MATCH (manager:Employee)-[:MANAGES]->(report:Employee)
RETURN manager.name as manager, manager.title as manager_title,
       report.name as report, report.title as report_title
ORDER BY manager.name
"""
results = run_query(query)
print(f"📊 Found {len(results)} management relationships:\n")
current_manager = None
for rel in results[:10]:  # Show first 10 for brevity
    if rel['manager'] != current_manager:
        current_manager = rel['manager']
        print(f"\n👤 {rel['manager']} ({rel['manager_title']}) manages:")
    print(f"   └─ {rel['report']} ({rel['report_title']})")
print()

In [ ]:
# 3️⃣ Find employees with specific skills
print("3️⃣ Who has TensorFlow skills?\n")

query = """
MATCH (e:Employee)-[:HAS_SKILL]->(s:Skill {name: "TensorFlow"})
RETURN e.name as name, e.title as title
ORDER BY e.name
"""
results = run_query(query)
print(f"📊 Found {len(results)} employees with TensorFlow skills:\n")
for emp in results:
    print(f"   🔧 {emp['name']} - {emp['title']}")
print()

In [ ]:
# 4️⃣ Find employees working on high-priority projects
print("4️⃣ Who's working on high-priority projects?\n")

query = """
MATCH (e:Employee)-[:ASSIGNED_TO]->(p:Project)
WHERE p.priority IN ['high', 'critical']
RETURN p.name as project, p.priority as priority, p.status as status,
       collect(e.name) as team_members
ORDER BY
    CASE p.priority
        WHEN 'critical' THEN 1
        WHEN 'high' THEN 2
        ELSE 3
    END
"""
results = run_query(query)
print(f"📊 Found {len(results)} high-priority projects:\n")
for proj in results:
    priority_emoji = "🔴" if proj['priority'] == 'critical' else "🟠"
    print(f"{priority_emoji} {proj['project']} [{proj['status']}]")
    print(f"   Team: {', '.join(proj['team_members'])}\n")
print()

In [ ]:
# 5️⃣ Find colleagues (people in same department)
print("5️⃣ Who are Sarah Chen's colleagues?\n")

query = """
MATCH (sarah:Employee {name: "Sarah Chen"})-[:WORKS_IN]->(d:Department)<-[:WORKS_IN]-(colleague:Employee)
WHERE sarah <> colleague
RETURN colleague.name as name, colleague.title as title, d.name as department
ORDER BY colleague.name
"""
results = run_query(query)
print(f"📊 Sarah Chen has {len(results)} colleagues in her department:\n")
for col in results:
    print(f"   👥 {col['name']} - {col['title']}")
print()

In [ ]:
# 6️⃣ Skill overlap analysis
print("6️⃣ Who shares the most skills with Sarah Chen?\n")

query = """
MATCH (sarah:Employee {name: "Sarah Chen"})-[:HAS_SKILL]->(s:Skill)<-[:HAS_SKILL]-(colleague:Employee)
WHERE sarah <> colleague
RETURN colleague.name as name,
       colleague.title as title,
       count(s) as shared_skills,
       collect(s.name) as skills
ORDER BY shared_skills DESC
LIMIT 5
"""
results = run_query(query)
print(f"📊 Top colleagues by shared skills with Sarah Chen:\n")
for i, person in enumerate(results, 1):
    print(f"{i}. {person['name']} ({person['title']})")
    print(f"   Shared skills ({person['shared_skills']}): {', '.join(person['skills'])}\n")
print()

In [ ]:
# 7️⃣ Project team composition
print("7️⃣ Who's on the 'Customer Churn Prediction' team?\n")

query = """
MATCH (e:Employee)-[:ASSIGNED_TO]->(p:Project {name: "Customer Churn Prediction"})
RETURN p.name as project,
       p.status as status,
       p.priority as priority,
       collect({name: e.name, title: e.title}) as team
"""
results = run_query(query)
if results:
    proj = results[0]
    print(f"📁 Project: {proj['project']}")
    print(f"   Status: {proj['status']} | Priority: {proj['priority']}\n")
    print(f"   👥 Team Members ({len(proj['team'])}):\n")
    for member in proj['team']:
        print(f"      • {member['name']} - {member['title']}")
print()

In [ ]:
# 8️⃣ Find paths between employees (relationship chains)
print("8️⃣ How is Sarah Chen connected to Carlos Santos?\n")

query = """
MATCH path = (sarah:Employee {name: "Sarah Chen"})-[*1..4]-(carlos:Employee {name: "Carlos Santos"})
RETURN [node in nodes(path) | node.name] as connection_path,
       [rel in relationships(path) | type(rel)] as relationship_types,
       length(path) as path_length
ORDER BY path_length
LIMIT 3
"""
results = run_query(query)
print(f"📊 Found {len(results)} connection paths:\n")
for i, path in enumerate(results, 1):
    print(f"Path {i} ({path['path_length']} degrees of separation):")
    for j in range(len(path['connection_path']) - 1):
        print(f"   {path['connection_path'][j]} --[{path['relationship_types'][j]}]--> ", end="")
    print(path['connection_path'][-1])
    print()
print()

In [ ]:
# 9️⃣ Department budget analysis with employee count
print("9️⃣ Department resource analysis:\n")

query = """
MATCH (d:Department)
OPTIONAL MATCH (e:Employee)-[:WORKS_IN]->(d)
RETURN d.name as department,
       d.budget as budget,
       count(e) as employee_count,
       d.budget / count(e) as budget_per_employee
ORDER BY d.budget DESC
"""
results = run_query(query)
print(f"📊 Department Resource Allocation:\n")
for dept in results:
    print(f"🏢 {dept['department']}")
    print(f"   Budget: ${dept['budget']:,}")
    print(f"   Employees: {dept['employee_count']}")
    print(f"   Budget per employee: ${dept['budget_per_employee']:,.0f}\n")
print()

In [ ]:
# 🔟 Find potential collaborators (similar skills + different projects)
print("🔟 Recommended collaborators for cross-team projects:\n")

query = """
MATCH (e1:Employee {name: "Priya Patel"})-[:HAS_SKILL]->(s:Skill)<-[:HAS_SKILL]-(e2:Employee)
MATCH (e1)-[:WORKS_IN]->(d1:Department), (e2)-[:WORKS_IN]->(d2:Department)
WHERE e1 <> e2 AND d1 <> d2
RETURN e2.name as name,
       e2.title as title,
       d2.name as department,
       count(DISTINCT s) as shared_skills,
       collect(DISTINCT s.name)[0..3] as sample_skills
ORDER BY shared_skills DESC
LIMIT 5
"""
results = run_query(query)
print(f"📊 Priya Patel should collaborate with:\n")
for i, person in enumerate(results, 1):
    print(f"{i}. {person['name']} from {person['department']}")
    print(f"   {person['title']}")
    print(f"   {person['shared_skills']} shared skills: {', '.join(person['sample_skills'])}\n")
print()

In [ ]:
# 1️⃣1️⃣ Multi-hop: Find employees by skill category, then their projects
print("1️⃣1️⃣ What projects are ML experts working on?\n")

query = """
MATCH (e:Employee)-[:HAS_SKILL]->(s:Skill)
WHERE s.category = "ML Framework"
MATCH (e)-[:ASSIGNED_TO]->(p:Project)
RETURN DISTINCT p.name as project,
       p.priority as priority,
       collect(DISTINCT e.name) as ml_experts
ORDER BY
    CASE p.priority
        WHEN 'critical' THEN 1
        WHEN 'high' THEN 2
        ELSE 3
    END
"""
results = run_query(query)
print(f"📊 Projects with ML Framework experts:\n")
for proj in results:
    print(f"📁 {proj['project']} [{proj['priority']}]")
    print(f"   ML Experts: {', '.join(proj['ml_experts'])}\n")
print()

In [ ]:
# 1️⃣2️⃣ Find the most in-demand skills
print("1️⃣2️⃣ Most in-demand skills across the company:\n")

query = """
MATCH (e:Employee)-[:HAS_SKILL]->(s:Skill)
RETURN s.name as skill,
       s.category as category,
       count(e) as employee_count,
       collect(e.name)[0..3] as sample_employees
ORDER BY employee_count DESC
LIMIT 10
"""
results = run_query(query)
print(f"📊 Top 10 Skills:\n")
for i, skill in enumerate(results, 1):
    print(f"{i}. {skill['skill']} ({skill['category']})")
    print(f"   {skill['employee_count']} employees have this skill")
    print(f"   Including: {', '.join(skill['sample_employees'])}...\n")
print()

---
## 📊 Part 6: Visualizing Your Graph with Neo4j Browser

### What is Neo4j Browser?

**Neo4j Browser** is a built-in, interactive web interface that comes with Neo4j. It's the professional tool for exploring, querying, and visualizing graph databases. Think of it as a graphical SQL client, but for graphs!

### 🎨 Features of Neo4j Browser

- **Interactive graph visualization**: Click nodes to explore, expand relationships, see properties
- **Built-in Cypher editor**: Write and execute queries with autocomplete
- **Result visualization**: View results as graphs, tables, or JSON
- **Query history**: Save and replay queries
- **Graph styling**: Customize colors, sizes, and labels

### 🌐 Accessing Neo4j Browser

#### **In a Local Neo4j Installation:**
When you run Neo4j on your computer (Neo4j Desktop or local server), you can access Neo4j Browser at:
```
http://localhost:7474/browser/
```

#### **In Google Colab (This Notebook):**

⚠️ **Important Limitation**: Neo4j Browser runs on `localhost:7474`, which isn't directly accessible from Colab since Colab runs on Google's servers, not your local machine.

**Your Options:**
1. **For this exercise**: Use the query-based exploration below (shows results as tables)
2. **For full visualization**: Install Neo4j Desktop on your computer and run these queries there
3. **Alternative**: Use Neo4j Aura (cloud-hosted) which provides browser access from anywhere

### 📋 Exploring Your Graph (Colab-Friendly Approach)

Since we can't use Neo4j Browser directly in Colab, let's explore the graph using queries that return structured results. These same queries work beautifully in Neo4j Browser when run locally!

In [ ]:
# 📊 Exploration 1: Overview of the entire graph

print("📊 Graph Overview: What's in our knowledge graph?\n")

query = """
MATCH (n)
WITH labels(n) as nodeType, count(*) as count
RETURN nodeType, count
ORDER BY count DESC
"""

results = run_query(query)
print("Node Types:")
print("-" * 40)
for row in results:
    print(f"  {row['nodeType'][0]}: {row['count']} nodes")

query = """
MATCH ()-[r]->()
WITH type(r) as relType, count(*) as count
RETURN relType, count
ORDER BY count DESC
"""

results = run_query(query)
print("\nRelationship Types:")
print("-" * 40)
for row in results:
    print(f"  {row['relType']}: {row['count']} relationships")

print("\n✅ This gives you a bird's-eye view of your graph structure!")
print("💡 In Neo4j Browser, you'd see this as an interactive visualization.")

In [ ]:
# 📊 Exploration 2: Data Science Department subgraph

print("📊 Exploring the Data Science Department structure\n")

query = """
MATCH (d:Department {name: "Data Science"})<-[:WORKS_IN]-(e:Employee)
OPTIONAL MATCH (e)-[:MANAGES]->(report:Employee)
RETURN e.name as employee,
       e.title as title,
       CASE WHEN report IS NOT NULL THEN report.name ELSE 'No direct reports' END as manages
ORDER BY e.years_experience DESC
"""

results = run_query(query)

print("Data Science Department Structure:")
print("=" * 80)
for row in results:
    print(f"\n👤 {row['employee']} ({row['title']})")
    if row['manages'] != 'No direct reports':
        print(f"   └─ Manages: {row['manages']}")

print("\n💡 In Neo4j Browser:")
print("   • You'd see this as an interactive graph with nodes and arrows")
print("   • Click any employee to see their full details")
print("   • Expand to see more relationships")

In [ ]:
# 📊 Exploration 3: Project team network

print("📊 Exploring the 'Fraud Detection System' project team\n")

query = """
MATCH (p:Project {name: "Fraud Detection System"})<-[:ASSIGNED_TO]-(e:Employee)
MATCH (e)-[:HAS_SKILL]->(s:Skill)
RETURN e.name as employee,
       e.title as title,
       collect(s.name) as skills
ORDER BY e.name
"""

results = run_query(query)

print("Fraud Detection System - Project Team:")
print("=" * 80)
for row in results:
    print(f"\n👤 {row['employee']} ({row['title']})")
    print(f"   🎯 Skills: {', '.join(row['skills'][:5])}")

# Show project details
query2 = """
MATCH (p:Project {name: "Fraud Detection System"})
RETURN p.status as status, p.priority as priority, p.budget as budget
"""
result2 = run_query(query2)[0]

print(f"\n📁 Project Details:")
print(f"   Status: {result2['status']}")
print(f"   Priority: {result2['priority']}")
print(f"   Budget: ${result2['budget']:,}")

print("\n💡 In Neo4j Browser:")
print("   • Nodes would be color-coded: Employees (blue), Project (red), Skills (purple)")
print("   • You could see all connections radiating from the central project node")

In [ ]:
# 📊 Exploration 4: Skill distribution across departments

print("📊 Python Skill Network - Who knows Python?\n")

query = """
MATCH (s:Skill {name: "Python"})<-[:HAS_SKILL]-(e:Employee)
MATCH (e)-[:WORKS_IN]->(d:Department)
RETURN d.name as department,
       collect(e.name) as employees,
       count(e) as employee_count
ORDER BY employee_count DESC
"""

results = run_query(query)

print("Python Skills by Department:")
print("=" * 80)
for row in results:
    print(f"\n🏢 {row['department']} ({row['employee_count']} employees)")
    for emp in row['employees']:
        print(f"   • {emp}")

print("\n💡 In Neo4j Browser:")
print("   • The 'Python' skill node would be at the center")
print("   • Lines connect it to all employees who have that skill")
print("   • Different colored nodes show which department each person is in")

In [ ]:
# 📊 Exploration 5: Cross-department collaboration

print("📊 Real-time Analytics Dashboard - Cross-Department Collaboration\n")

query = """
MATCH (p:Project {name: "Real-time Analytics Dashboard"})<-[:ASSIGNED_TO]-(e:Employee)
MATCH (e)-[:WORKS_IN]->(d:Department)
RETURN d.name as department,
       collect({name: e.name, title: e.title}) as team_members,
       count(e) as count
ORDER BY count DESC
"""

results = run_query(query)

print("Cross-Department Collaboration on Analytics Dashboard:")
print("=" * 80)

total_people = sum(r['count'] for r in results)
print(f"\n📊 Total team size: {total_people} people across {len(results)} departments\n")

for row in results:
    print(f"\n🏢 {row['department']} Department ({row['count']} people)")
    for member in row['team_members']:
        print(f"   • {member['name']} - {member['title']}")

print("\n💡 In Neo4j Browser:")
print("   • You'd see the project node in the center")
print("   • Employee nodes from different departments clustered around it")
print("   • Department nodes showing organizational structure")
print("   • This instantly reveals cross-functional collaboration patterns!")

### 🎯 Neo4j Browser - Key Takeaways

**What We Learned:**
- Neo4j Browser is a powerful built-in visualization tool for exploring graph databases
- It provides interactive, clickable graph visualizations
- In Colab, we can't access it directly (since it runs on localhost:7474)
- The same Cypher queries work in both environments—only the visualization differs

**When to Use Neo4j Browser:**
- ✅ **Exploring data**: Visually discover patterns and relationships
- ✅ **Debugging queries**: See exactly what your Cypher returns
- ✅ **Presentations**: Show stakeholders how data is connected
- ✅ **Development**: Build and test queries interactively

**How to Get Full Visualization Access:**
1. **Install Neo4j Desktop** (free): Download from [neo4j.com/download](https://neo4j.com/download/)
2. **Use Neo4j Aura** (cloud-based): Sign up at [neo4j.com/aura](https://neo4j.com/aura)
3. **Run locally**: Follow the same installation steps from this notebook on your machine

**💡 Pro Tip:** All the queries we ran above produce beautiful interactive graphs in Neo4j Browser. Try them locally to see the full visual experience!

---

Now let's move on to best practices for working with graph databases.

### 📊 Matplotlib Visualizations for Colab

Since we don't have direct access to Neo4j Browser in Colab, let's create **matplotlib visualizations** to see our graph structure! We'll use NetworkX to create graph layouts and matplotlib to render them with color-coded nodes.

**What we'll visualize:**
- Department organizational structure
- Project team networks
- Skill distribution across employees
- Cross-department collaboration patterns

Let's create our visualization function! 🎨

In [ ]:
# 🎨 Create visualization function using NetworkX and Matplotlib

def visualize_subgraph(query: str, title: str, node_colors: dict = None):
    """
    Execute a Cypher query and visualize the resulting subgraph.

    Args:
        query: Cypher query that returns nodes and relationships
        title: Title for the visualization
        node_colors: Dictionary mapping node labels to colors
    """
    # Default color scheme
    if node_colors is None:
        node_colors = {
            'Employee': '#3498db',    # Blue
            'Department': '#e74c3c',  # Red
            'Project': '#2ecc71',     # Green
            'Skill': '#f39c12'        # Orange
        }

    # Execute query to get graph data
    with driver.session() as session:
        result = session.run(query)

        # Create NetworkX graph
        G = nx.DiGraph()

        # Process results
        for record in result:
            # Extract nodes and relationships from the result
            for key in record.keys():
                value = record[key]

                # Handle nodes
                if hasattr(value, 'labels') and hasattr(value, 'id'):
                    node_id = value.id
                    label = list(value.labels)[0] if value.labels else 'Unknown'
                    name = value.get('name', value.get('title', f'Node{node_id}'))
                    G.add_node(node_id, label=label, name=name, color=node_colors.get(label, '#95a5a6'))

                # Handle relationships in path format
                if hasattr(value, 'nodes') and hasattr(value, 'relationships'):
                    # This is a path object
                    for node in value.nodes:
                        node_id = node.id
                        label = list(node.labels)[0] if node.labels else 'Unknown'
                        name = node.get('name', node.get('title', f'Node{node_id}'))
                        G.add_node(node_id, label=label, name=name, color=node_colors.get(label, '#95a5a6'))

                    for rel in value.relationships:
                        G.add_edge(rel.start_node.id, rel.end_node.id, type=rel.type)

    if len(G.nodes()) == 0:
        print(f"⚠️ No data returned from query for '{title}'")
        return

    # Create visualization
    plt.figure(figsize=(14, 10))

    # Use spring layout for better visualization
    pos = nx.spring_layout(G, k=2, iterations=50, seed=42)

    # Draw nodes by type with different colors
    for label_type, color in node_colors.items():
        nodes_of_type = [n for n, d in G.nodes(data=True) if d.get('label') == label_type]
        if nodes_of_type:
            nx.draw_networkx_nodes(G, pos, nodelist=nodes_of_type,
                                 node_color=color, node_size=800, alpha=0.9, label=label_type)

    # Draw edges
    nx.draw_networkx_edges(G, pos, edge_color='#7f8c8d', arrows=True,
                          arrowsize=15, alpha=0.5, width=1.5, arrowstyle='->')

    # Draw labels
    labels = {n: d['name'][:15] for n, d in G.nodes(data=True)}  # Truncate long names
    nx.draw_networkx_labels(G, pos, labels, font_size=8, font_weight='bold')

    # Add legend and title
    plt.title(title, fontsize=16, fontweight='bold', pad=20)
    plt.legend(scatterpoints=1, loc='upper left', fontsize=10)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    print(f"✅ Visualization complete: {len(G.nodes())} nodes, {len(G.edges())} relationships\n")

print("✅ Visualization function created!")

In [ ]:
# 📊 Visualization 1: Data Science Department Structure

print("📊 Visualizing Data Science Department...\n")

query = """
MATCH path = (d:Department {name: "Data Science"})<-[:WORKS_IN]-(e:Employee)
OPTIONAL MATCH (e)-[r:MANAGES]->(report:Employee)
RETURN path
LIMIT 30
"""

visualize_subgraph(query, "Data Science Department - Organizational Structure")

In [ ]:
# 📊 Visualization 2: Fraud Detection Project Team

print("📊 Visualizing Fraud Detection System project team...\n")

query = """
MATCH path = (p:Project {name: "Fraud Detection System"})<-[:ASSIGNED_TO]-(e:Employee)
OPTIONAL MATCH (e)-[:HAS_SKILL]->(s:Skill)
RETURN path
LIMIT 30
"""

visualize_subgraph(query, "Fraud Detection System - Project Team & Skills")

In [ ]:
# 📊 Visualization 3: Python Skill Network

print("📊 Visualizing Python skill network...\n")

query = """
MATCH path = (s:Skill {name: "Python"})<-[:HAS_SKILL]-(e:Employee)-[:WORKS_IN]->(d:Department)
RETURN path
LIMIT 25
"""

visualize_subgraph(query, "Python Skill Distribution Across Departments")

In [ ]:
# 📊 Visualization 4: Cross-Department Collaboration

print("📊 Visualizing cross-department collaboration on Analytics Dashboard...\n")

query = """
MATCH path = (p:Project {name: "Real-time Analytics Dashboard"})<-[:ASSIGNED_TO]-(e:Employee)-[:WORKS_IN]->(d:Department)
RETURN path
LIMIT 20
"""

visualize_subgraph(query, "Real-time Analytics Dashboard - Cross-Department Collaboration")

In [ ]:
# 📊 Visualization 5: Management Hierarchy

print("📊 Visualizing management hierarchy from CEO...\n")

query = """
MATCH path = (ceo:Employee {title: "CEO"})-[:MANAGES*1..3]->(report:Employee)
RETURN path
LIMIT 25
"""

visualize_subgraph(query, "Organizational Management Hierarchy - Top 3 Levels")

### 🎯 Visualization Key Takeaways

**What These Visualizations Show:**
- **Color-coded nodes**: Each entity type (Employee, Department, Project, Skill) has its own color
- **Directed relationships**: Arrows show the direction of relationships (e.g., Employee → Department)
- **Network patterns**: See how entities cluster and connect across the organization
- **Graph structure**: Understand the shape and density of your knowledge graph

**Insights from Our Visualizations:**
1. **Department Structure**: Shows how employees are organized and who manages whom
2. **Project Teams**: Reveals which employees work together and what skills they bring
3. **Skill Distribution**: Identifies expertise across departments (e.g., Python is widely distributed)
4. **Cross-Department Work**: Highlights collaboration patterns between departments
5. **Management Hierarchy**: Shows reporting chains from leadership down

**💡 Comparison to Neo4j Browser:**
- These matplotlib visualizations work great in Colab and show the graph structure clearly
- Neo4j Browser adds interactivity: click nodes to expand, filter, and explore dynamically
- Both approaches are valuable: matplotlib for static analysis, Neo4j Browser for interactive exploration

**Next Steps:**
For production applications, consider:
- **Plotly** or **Pyvis** for interactive web-based graph visualizations
- **Neo4j Bloom** for business-user-friendly graph exploration
- **Graph Data Science library** for advanced analytics (centrality, community detection, etc.)

---

Now you can see your graph database visually! Let's wrap up with next steps.

---
## 🚀 Part 7: Next Steps and Extensions

### 🔜 What's Next: Notebook 2 - GraphRAG

In the next notebook, you'll take your graph database skills to the next level by combining them with:

- **Vector embeddings** for semantic search
- **LlamaIndex** for document ingestion and retrieval
- **LLMs** for entity extraction and query generation
- **Hybrid search** that leverages both graph relationships and vector similarity

You'll learn how **GraphRAG** (Graph + Retrieval-Augmented Generation) provides better context and reduces hallucinations compared to traditional RAG systems.


### 📖 Additional Resources

- **Neo4j Documentation**: [neo4j.com/docs](https://neo4j.com/docs/)
- **Cypher Manual**: [neo4j.com/docs/cypher-manual](https://neo4j.com/docs/cypher-manual/current/)
- **Neo4j Graph Academy**: Free online courses at [neo4j.com/graphacademy](https://neo4j.com/graphacademy/)
- **Graph Data Science Library**: Advanced algorithms for centrality, community detection, and more



In [ ]:
# 🧹 Optional: Clean up and close connections

print("🧹 Closing Neo4j driver connection...")
driver.close()
print("✅ Connection closed successfully!")
print("\n👋 Thanks for learning graph databases with Neo4j!")
print("📚 See you in Notebook 2: GraphRAG with LlamaIndex! 🚀")